In [0]:
dbutils.widgets.text("p_data_date", "")
v_data_date = dbutils.widgets.get("p_data_date")

In [0]:
dbutils.secrets.list(scope='fomular1-scope') 

In [0]:
formular1_key = dbutils.secrets.get(scope='fomular1-scope', key='formuladl1-key-account')

In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", formular1_key)

In [0]:
races_df = spark.read.option("header", True).option("inferSchema", True).csv(f"abfss://raw@fomular1dl1.dfs.core.windows.net/{v_data_date}/races.csv")

In [0]:
races_df.show()

In [0]:
races_df.printSchema()

In [0]:
rename_df = races_df.select("raceId", "year", "round", "circuitId", "name", "date", "time")


In [0]:
from pyspark.sql.functions import col, lit, current_timestamp, to_timestamp, concat

rename_df = rename_df.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("year", "race_year") \
.withColumnRenamed("circuitId", "circuit_id") \
.withColumn("ingestion_date", current_timestamp()) \
.withColumn('race_timestamp', to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))
display(rename_df)


In [0]:


rename_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.races")

In [0]:
spark.read.parquet("abfss://processed@fomular1dl1.dfs.core.windows.net/races").show()